<a href="https://colab.research.google.com/github/Toadoum/STEM-Away-Job/blob/master/FineTune_BERT_Model_STEM_Away.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/My Drive/STEMAway/train_skills_labeled1 - train_skills_labeled.csv', error_bad_lines = True)

In [ ]:
data['skill'].value_counts()

0    31746
1     1376
Name: skill, dtype: int64

In [ ]:
data = data[['phrase','skill']]
data=data[0:1000]
data.head()

,phrase,skill
0,deliverables,0
1,a broad variety,0
2,manage ml infrastructures,1
3,and/or patent,0
4,sellers,0


In [ ]:
data['skill'].value_counts()

0    728
1    272
Name: skill, dtype: int64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model = model.to('cuda')

In [ ]:
sample_data = ["We need someone who know python","Django is a must "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 2057, 2342, 2619, 2040, 2113, 18750, 102], [101, 6520, 23422, 2003, 1037, 2442, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0]]}

In [ ]:
X = list(data["phrase"])
y = list(data["skill"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
len(X_train),len(X_val)

(800, 200)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

{'input_ids': tensor([ 101, 3813, 1011, 2898, 7904, 6666,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor(0)}

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=4,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 800
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 400
  Number of trainable parameters = 109483778


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=400, training_loss=0.330270881652832, metrics={'train_runtime': 40.7931, 'train_samples_per_second': 78.445, 'train_steps_per_second': 9.806, 'total_flos': 37822214208000.0, 'train_loss': 0.330270881652832, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.6579039096832275,
 'eval_accuracy': 0.86,
 'eval_precision': 0.7721518987341772,
 'eval_recall': 0.7176470588235294,
 'eval_f1': 0.7439024390243902,
 'eval_runtime': 0.528,
 'eval_samples_per_second': 568.211,
 'eval_steps_per_second': 71.973,
 'epoch': 4.0}

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
text = "Machine Learning"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-2.0937,  2.9042]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.0067, 0.9933]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.00670693, 0.99329305]], dtype=float32)

In [ ]:
trainer.save_model('CustomModel')

Saving model checkpoint to CustomModel
Configuration saved in CustomModel/config.json
Model weights saved in CustomModel/pytorch_model.bin


In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2.to('cuda')

loading configuration file CustomModel/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file CustomModel/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were ini

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# text = "That was good point"
text = "bonjour"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.9944635 , 0.00553646]], dtype=float32)

In [ ]:
#predictions = 1 if predictions[0][1] >= 0.5 else 0

# **How we can Automate this**

In [ ]:
# as if input was given on website 
jd = input('Paste job description here');

Paste job description hereC3.ai is a leading enterprise AI software provider for accelerating digital transformation. The comprehensive and proven C3 AI Suite uses a model-driven abstraction layer to enable organizations to develop, deploy, and operate enterprise scale AI applications 40x to 100x faster than alternative approaches. www.c3.ai  As a Data Scientist, you will participate in the definition of new analytics capabilities able to provide our customers with the information they need to make proper decisions to support our customers in operating the internet of things (IoT). In addition, you will help find the appropriate machine learning / data mining algorithms to answer these questions. Finally, you will be responsible for implementing this into the product and making it available to our customers.  Qualified candidates will have an in-depth knowledge of most common machine learning techniques and their application. You will also understand the limitations of these algorithms

In [ ]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
# input must be tokenized and pos tagging is done with vamnilla nltk pos tag
import nltk
tokens = nltk.word_tokenize(jd.lower())
tagged = nltk.pos_tag(tokens)
tagged

[('c3.ai', 'NN'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('leading', 'VBG'),
 ('enterprise', 'NN'),
 ('ai', 'NN'),
 ('software', 'NN'),
 ('provider', 'NN'),
 ('for', 'IN'),
 ('accelerating', 'VBG'),
 ('digital', 'JJ'),
 ('transformation', 'NN'),
 ('.', '.'),
 ('the', 'DT'),
 ('comprehensive', 'JJ'),
 ('and', 'CC'),
 ('proven', 'JJ'),
 ('c3', 'NN'),
 ('ai', 'VBP'),
 ('suite', 'JJ'),
 ('uses', 'VBZ'),
 ('a', 'DT'),
 ('model-driven', 'JJ'),
 ('abstraction', 'NN'),
 ('layer', 'NN'),
 ('to', 'TO'),
 ('enable', 'VB'),
 ('organizations', 'NNS'),
 ('to', 'TO'),
 ('develop', 'VB'),
 (',', ','),
 ('deploy', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('operate', 'VBP'),
 ('enterprise', 'NN'),
 ('scale', 'NN'),
 ('ai', 'VBP'),
 ('applications', 'NNS'),
 ('40x', 'CD'),
 ('to', 'TO'),
 ('100x', 'CD'),
 ('faster', 'JJR'),
 ('than', 'IN'),
 ('alternative', 'JJ'),
 ('approaches', 'NNS'),
 ('.', '.'),
 ('www.c3.ai', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('data', 'NN'),
 ('scientist', 'NN'),
 (',', ','),
 ('you', 'PRP'

## Extraction of identified POS patterns

In [ ]:
# noun phrase
# first pattern for extraction
# an optional determinate followed by any number of adjectives and a noun, plural noun or proper noun
grammar1 = ('''Noun Phrases: {<DT>?<JJ>*<NN|NNS|NNP>+}''')
chunkParser = nltk.RegexpParser(grammar1)
tree1 = chunkParser.parse(tagged)

# typical noun phrase pattern appending to be concatted later
g1_chunks = []
for subtree in tree1.subtrees(filter=lambda t: t.label() == 'Noun Phrases'):
    # print(subtree)
    g1_chunks.append(subtree)

In [ ]:
# Noun phrase variation; 
# second pattern for extraction
# preposition maybe, any number of adjective or nouns, any plural nouns or singular nouns
grammar2 = ('''NP2: {<IN>?<JJ|NN>*<NNS|NN>} ''')
chunkParser = nltk.RegexpParser(grammar2)
tree2 = chunkParser.parse(tagged)

# variation of a noun phrase pattern to be pickled for later analyses
g2_chunks = []
for subtree in tree2.subtrees(filter=lambda t: t.label() == 'NP2'):
    # print(subtree)
    g2_chunks.append(subtree)

In [ ]:
# third pattern for extraction
# any sort of verb followed by any number of nouns
grammar3 = ('''
    VS: {<VBG|VBZ|VBP|VBD|VB|VBN><NNS|NN>*}
    ''')
chunkParser = nltk.RegexpParser(grammar3)
tree3 = chunkParser.parse(tagged)

# verb-noun pattern appending to be concatted later
g3_chunks = []
for subtree in tree3.subtrees(filter=lambda t: t.label() == 'VS'):
    # print(subtree)
    g3_chunks.append(subtree)

In [ ]:
# fourth pattern for extraction
# any number of a singular or plural noun followed by a comma followed by the same noun, noun, noun pattern
grammar4 = ('''
    Commas: {<NN|NNS>*<,><NN|NNS>*<,><NN|NNS>*} 
    ''')
chunkParser = nltk.RegexpParser(grammar4)
tree4 = chunkParser.parse(tagged)

# common pattern of listing skills appending to be concatted later
g4_chunks = []
for subtree in tree4.subtrees(filter=lambda t: t.label() == 'Commas'):
    # print(subtree)
    g4_chunks.append(subtree)

In [ ]:
#sample size reused from prior cell use just in case a sample would be 
# used to reduce the amount of extractions to go through
print('Length:', len(g1_chunks), 'Sample Size:', len(g1_chunks) * .10)
print('Length:', len(g2_chunks), 'Sample Size:', len(g2_chunks) * .10) 
print('Length:', len(g3_chunks), 'Sample Size:', len(g3_chunks) * .10)
print('Length:', len(g4_chunks), 'Sample Size:', len(g4_chunks) * .10)

Length: 119 Sample Size: 11.9
Length: 122 Sample Size: 12.200000000000001
Length: 67 Sample Size: 6.7
Length: 5 Sample Size: 0.5


## Chunk Dataframe

In [ ]:
def data_process(chunks):
    '''creates a dataframe that easily parses the chunks data 
    '''
    df = pd.DataFrame(chunks)    
    df.fillna('X', inplace = True)
    
    pattern = []
    for row in df.values:
        phrase = ''
        for tup in row:
            # needs a space at the end for seperation
            phrase += tup[0] + ' '
        phrase = ''.join(phrase)
        # could use padding tages but encoder method will provide during 
        # tokenizing/embeddings; X can replace paddding for now
        pattern.append( phrase.replace('X', '').strip())
    df['phrase'] = pattern
    # only returns 10% of each dataframe to be used 
    return df.phrase

In [ ]:
#one training corpus with 10% of each POS regex identification
chunked = pd.concat([data_process(g1_chunks),
                      data_process(g2_chunks), 
                      data_process(g3_chunks),
                      data_process(g4_chunks)], 
                        ignore_index = True )

In [ ]:
unseen = np.array(chunked)
unseen_text = unseen[:, np.newaxis]

In [ ]:
len(unseen_text)

313

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2.to('cuda')

loading configuration file CustomModel/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file CustomModel/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were ini

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
predictions_list = []
list_word=[]
for i in range(len(unseen_text)):
    for j in unseen_text[i]:
        text=j
        list_word.append(j)
        #list_word.append(j)
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to('cuda')
        outputs = model_2(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predictions = predictions.cpu().detach().numpy()
        prediction = 1 if predictions[0][1] >= 0.5 else 0
        predictions_list.append(prediction)
        #list_word.append(text)
print(len(list_word))
print(len(predictions_list))
import pandas as pd

result = {'Word': list_word, 'Prediction': predictions_list}
result_df = pd.DataFrame(result)

313
313


In [ ]:
result_df.head(20)

,Word,Prediction
0,c3.ai,1
1,enterprise ai software provider,1
2,digital transformation,1
3,proven c3,1
4,a model-driven abstraction layer,1
5,organizations,0
6,deploy,1
7,enterprise scale,0
8,applications,0
9,alternative approaches,0


313
626


ValueError: ignored

In [ ]:
result_df.head()

,Word,Prediction
0,"github , papers ,",1
1,"github , papers ,",1
2,"github , papers ,",1
3,"github , papers ,",1
4,"github , papers ,",1
5,"github , papers ,",0
6,"github , papers ,",1
7,"github , papers ,",0
8,"github , papers ,",0
9,"github , papers ,",0
